Ngram Model

In [7]:
# code courtesy of https://nlpforhackers.io/language-models/
from nltk.corpus import reuters
# from nltk import bigrams, trigrams
from nltk import ngrams
from collections import Counter, defaultdict

import itertools 


In [8]:
def getAlarmsSeqs(fname):
    lines = None
    with open(fname) as f:
        lines = f.readlines()

    temp_alarms_seqs = []
    for i,line in enumerate(lines):
        seq = line.split()
        temp_alarms_seqs.append(seq)
    
    temp_alarms_seqs =  list(itertools.chain(*temp_alarms_seqs)) 


    return temp_alarms_seqs

fname = "../.data/train.tokens"
train_alarms_seqs = getAlarmsSeqs(fname)

# train_alarms_seqs[0:50]
      

In [9]:


# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

for a1, a2, a3,a4, a5, a6  in ngrams(train_alarms_seqs, n=6, pad_right=True, pad_left=True):
    # print(a1,a2,a3,a4,a5)
    model[(a1, a2, a3,a4, a5)][a6] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [10]:
d = dict(model[ '47TI2086',
 '47PDI003',
 '47TI3418',
 '47LI003',
 '47LI003'])

print(list(d.keys())[0],list(d.values())[0]) 

print(d)

47XL015-ANN 1.0
{'47XL015-ANN': 1.0}


In [11]:
# calculate the accuracy
fname = "../.data/val.tokens"
val_alarms_seqs = getAlarmsSeqs(fname)
alarm2acc = {}

for a in train_alarms_seqs:
    alarm2acc[a] = {"true":0, "false":0, "acc":0}

for a in val_alarms_seqs:
    alarm2acc[a] = {"true":0, "false":0, "acc":0}
 


for i in range(len(val_alarms_seqs)-6):
    seq = train_alarms_seqs[i:i+6]
    seq_in = seq[0:5]
    y_true = seq[5]
    # print(seq_in,y_true)
    
    d = dict(model[seq_in[0],seq_in[1],seq_in[2],seq_in[3],seq_in[4]])
    y_pred = list(d.keys())[0] 
    if y_pred == y_true:
       alarm2acc[y_true]['true'] += 1
    else:
       alarm2acc[y_true]['false'] += 1  
        # print(list(d.keys())[0],list(d.values())[0]) 


for a in alarm2acc.keys():
    if alarm2acc[a]["true"]+alarm2acc[a]["false"]>0: 
        alarm2acc[a]["acc"] =alarm2acc[a]["true"] /(alarm2acc[a]["true"]+alarm2acc[a]["false"]) 


alarm2acc


    




{'47TI934I': {'true': 258, 'false': 86, 'acc': 0.75},
 '47FIC012': {'true': 421, 'false': 102, 'acc': 0.8049713193116634},
 '47LIC005': {'true': 108, 'false': 17, 'acc': 0.864},
 '48TIC034': {'true': 592, 'false': 499, 'acc': 0.5426214482126489},
 '19LIC102': {'true': 29, 'false': 6, 'acc': 0.8285714285714286},
 '47TI2086': {'true': 59, 'false': 20, 'acc': 0.7468354430379747},
 '47PDI003': {'true': 736, 'false': 516, 'acc': 0.5878594249201278},
 '47TI3418': {'true': 54, 'false': 7, 'acc': 0.8852459016393442},
 '47LI003': {'true': 1676, 'false': 504, 'acc': 0.7688073394495413},
 '47XL015-ANN': {'true': 905, 'false': 139, 'acc': 0.8668582375478927},
 '47XL016-ANN': {'true': 885, 'false': 153, 'acc': 0.8526011560693642},
 '47LIC008': {'true': 314, 'false': 150, 'acc': 0.6767241379310345},
 '47LIC1506': {'true': 558, 'false': 229, 'acc': 0.7090216010165185},
 '48XL001-ANN': {'true': 1540, 'false': 1433, 'acc': 0.5179952909519004},
 '47LIC3408': {'true': 473, 'false': 200, 'acc': 0.70282317

In [12]:
# import random

# # starting words
# text = ["today", "the"]
# sentence_finished = False
 
# while not sentence_finished:
#   # select a random probability threshold  
#   r = random.random()
#   accumulator = .0

#   for word in model[tuple(text[-2:])].keys():
#       accumulator += model[tuple(text[-2:])][word]
#       # select words that are above the probability threshold
#       if accumulator >= r:
#           text.append(word)
#           break

#   if text[-2:] == [None, None]:
#       sentence_finished = True
 
# print (' '.join([t for t in text if t]))